# 통신사 고객 이탈 예측
https://www.kaggle.com/blastchar/telco-customer-churn


## 경쟁 설명
고객 유지를 위해 관련 고객 데이터를 분석하고 고객 보존프로그램을 개발

## 과정

1. 준비  
  1.1 라이브러리 임포트  
  1.2 데이터 가져오기
2. 데이터 탐색 및 전처리  
  2.1 각 요소의 유형,분포 파악 및 유형 코딩  
  2.2 결측치 확인 및 처리   
  2.3 이상치 확인 및 처리  
  2.4 시각화  
  2.5 상관관계 확인   
3. 데이터 모델링  
  3.1 의사결정나무 모델 적용  
  3.2 랜덤포레스트 적용  
  3.3 로지스틱 회귀 모델 적용
5. 평가

### 1. 준비
#### 1.1 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np

#시각화를 위한 라이브러리
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#### 1.2 데이터 가져오기


In [ ]:
df=pd.read_csv("/kaggle/input/WA_Fn-UseC_-Telco-Customer-Churn.csv")

### 2. 데이터 탐색

#### 2.1 각 요소의 유형,분포 파악 및 유형 코딩

In [ ]:
df.head()

**customerID**	고객 ID  
**gender** 성별 (여성, 남성)  
**SeniorCitizen**	노약자 여부 (1, 0)  
**Partner**	파트너가 있는지 여부 (예, 아니오)  
**Dependents**	부양 가족 여부 (예, 아니오)  
**tenure**	고객이 회사에 머물렀던 개월 수  
**PhoneService**	전화 서비스가 있는지 여부 (예, 아니요)  
**MultipleLines**	여러 회선을 사용하는지 여부 (예, 아니요, 전화 서비스 없음)  
**InternetService**	고객의 인터넷 서비스 제공 업체 (DSL, 광섬유, 아니요)  
**OnlineSecurity**	고객의 온라인 보안 여부 (예, 아니오, 인터넷 서비스 없음)  
**OnlineBackup**	고객이 온라인 백업을했는지 여부 (예, 아니요, 인터넷 서비스 없음)  
**DeviceProtection**	고객에게 기기 보호 기능이 있는지 여부 (예, 아니요, 인터넷 서비스 없음)  
**TechSupport**	기술 지원을 받았는지 여부 (예, 아니오, 인터넷 서비스 없음)  
**StreamingTV**	스트리밍 TV 보유 여부 (예, 아니요, 인터넷 서비스 없음)  
**StreamingMovies**	고객의 영화 스트리밍하 여부 (예, 아니요, 인터넷 서비스 없음)  
**Contract**	고객의 계약 기간 (월간, 1 년, 2 년)  
**PaperlessBilling**	종이없는 청구서 수신 여부 (예, 아니오)  
**PaymentMethod**	결제 수단 (전자 수표, 우편 수표, 은행 송금 (자동), 신용 카드 (자동))  
**MonthlyCharges**	매월 고객에게 청구되는 금액  
**TotalCharges**	고객에게 청구 된 총 금액  
**Churn**	이탈 여부 (예 또는 아니오)  

In [ ]:
df.dtypes

SeniorCitizen가 범주형 변수이나 int64 형태로 존재  
TotlaCharges는 수치형 변수이나 object 형태로 존재  
-> 각각에 적합한 형태로 변경 필요


In [ ]:
c=0
for i in df['TotalCharges']:
    c=c+1
    if i==" ":
        print(c,i)

TotalCharges의 값에 결측치가 공백' ' 형태로 존재하여 범주형태로 변경 불가  
-> 결측치가 있는 요소들은 유지개월이 0이므로 결측치를 0으로 대체

In [ ]:
df['TotalCharges']=df['TotalCharges'].replace(" ",0)
df['TotalCharges']=df['TotalCharges'].astype('float64')
df['SeniorCitizen']=df['SeniorCitizen'].astype('object')

In [ ]:
df.loc[df['gender'] =='Male', 'gender'] = "0"
df.loc[df['gender'] =='Female', 'gender'] = "1"

df.loc[df['Partner']=='Yes','Partner']="1"
df.loc[df['Partner']=='No','Partner']="0"

df.loc[df['Dependents']=='Yes','Dependents']="1"
df.loc[df['Dependents']=='No','Dependents']="0"

df.loc[df['PhoneService']=='Yes','PhoneService']="1"
df.loc[df['PhoneService']=='No','PhoneService']="0"

df.loc[df['MultipleLines']=='Yes','MultipleLines']="1"
df.loc[df['MultipleLines']=='No','MultipleLines']="0"
df.loc[df['MultipleLines']=='No phone service','MultipleLines']="2"

df.loc[df['InternetService']=='DSL','InternetService']="1"
df.loc[df['InternetService']=='Fiber optic','InternetService']="2"
df.loc[df['InternetService']=='No','InternetService']="0"

df.loc[df['OnlineBackup']=='Yes','OnlineBackup']="1"
df.loc[df['OnlineBackup']=='No','OnlineBackup']="0"
df.loc[df['OnlineBackup']=='No internet service','OnlineBackup']="2"

df.loc[df['OnlineSecurity']=='Yes','OnlineSecurity']="1"
df.loc[df['OnlineSecurity']=='No','OnlineSecurity']="0"
df.loc[df['OnlineSecurity']=='No internet service','OnlineSecurity']="2"

df.loc[df['DeviceProtection']=='Yes','DeviceProtection']="1"
df.loc[df['DeviceProtection']=='No','DeviceProtection']="0"
df.loc[df['DeviceProtection']=='No internet service','DeviceProtection']="2"

df.loc[df['TechSupport']=='Yes','TechSupport']="1"
df.loc[df['TechSupport']=='No','TechSupport']="0"
df.loc[df['TechSupport']=='No internet service','TechSupport']="2"

df.loc[df['StreamingTV']=='Yes','StreamingTV']="1"
df.loc[df['StreamingTV']=='No','StreamingTV']="0"
df.loc[df['StreamingTV']=='No internet service','StreamingTV']="2"

df.loc[df['StreamingMovies']=='Yes','StreamingMovies']="1"
df.loc[df['StreamingMovies']=='No','StreamingMovies']="0"
df.loc[df['StreamingMovies']=='No internet service','StreamingMovies']="2"

df.loc[df['Contract']=='Month-to-month','Contract']="0"
df.loc[df['Contract']=='One year','Contract']="1"
df.loc[df['Contract']=='Two year','Contract']="2"

df.loc[df['PaperlessBilling'] =='Yes', 'PaperlessBilling'] = "0"
df.loc[df['PaperlessBilling'] =='No', 'PaperlessBilling'] = "1"

df.loc[df['PaymentMethod'] =='Electronic check', 'PaymentMethod'] = "0"
df.loc[df['PaymentMethod'] =='Mailed check', 'PaymentMethod'] = "1"
df.loc[df['PaymentMethod'] =='Bank transfer (automatic)', 'PaymentMethod'] = "2"
df.loc[df['PaymentMethod'] =='Credit card (automatic)', 'PaymentMethod'] = "3"

df.loc[df['Churn']=='Yes','Churn']="1"
df.loc[df['Churn']=='No','Churn']="0"

#### 2.2 결측치 확인

In [ ]:
df.isnull().sum()

결측치 처리 완료

#### 2.3 이상치 확인 및 처리

In [ ]:
sns.boxplot(df['tenure']);

In [ ]:
sns.boxplot(df['MonthlyCharges']);

In [ ]:
sns.boxplot(df['TotalCharges']);

### 2.4 시각화

#### 범주형 변수

In [ ]:
def barplot_percentages(feature, orient='v', axis_name="percentage of customers"):
    ratios = pd.DataFrame()
    g = df.groupby(feature)["Churn"].value_counts().to_frame()
    g = g.rename({"Churn": axis_name}, axis=1).reset_index()
    g[axis_name] = g[axis_name]/len(df)
    if orient == 'v':
        ax = sns.barplot(x=feature, y= axis_name, hue='Churn', data=g, orient=orient)
        ax.set_yticklabels(['{:,.0%}'.format(y) for y in ax.get_yticks()])
    else:
        ax = sns.barplot(x= axis_name, y=feature, hue='Churn', data=g, orient=orient)
        ax.set_xticklabels(['{:,.0%}'.format(x) for x in ax.get_xticks()])
    ax.plot()
barplot_percentages("SeniorCitizen")

In [ ]:
df.head()
df.dtypes

In [ ]:
g = sns.FacetGrid(df, row='SeniorCitizen', col="gender", hue="Churn")
g.map(plt.scatter, "tenure", "MonthlyCharges", alpha=0.6)
g.add_legend();

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(12,4))
axis[0].set_title("Has partner")
axis[1].set_title("Has dependents")
axis_y = "percentage of customers"
# Plot Partner column
gp_partner = df.groupby('Partner')["Churn"].value_counts()/len(df)
gp_partner = gp_partner.to_frame().rename({"Churn": axis_y}, axis=1).reset_index()
ax = sns.barplot(x='Partner', y= axis_y, hue='Churn', data=gp_partner, ax=axis[0])
# Plot Dependents column
gp_dep = df.groupby('Dependents')["Churn"].value_counts()/len(df)
gp_dep = gp_dep.to_frame().rename({"Churn": axis_y}, axis=1).reset_index()
ax = sns.barplot(x='Dependents', y= axis_y, hue='Churn', data=gp_dep, ax=axis[1])

In [ ]:
plt.figure(figsize=(9, 4.5))
barplot_percentages("MultipleLines", orient='h')

In [ ]:
cols = ["OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies"]
df1 = pd.melt(df[df["InternetService"] != "No"][cols]).rename({'value': 'Has service'}, axis=1)
plt.figure(figsize=(10, 4.5))
ax = sns.countplot(data=df1, x='variable', hue='Has service')
ax.set(xlabel='Additional service', ylabel='Num of customers')
plt.show()

#### 숫자형 변수

In [ ]:
def kdeplot(feature):
    plt.figure(figsize=(9, 4))
    plt.title("KDE for {}".format(feature))
    ax0 = sns.kdeplot(df[df['Churn'] == '0'][feature].dropna(), color= 'navy', label= 'Churn: No')
    ax1 = sns.kdeplot(df[df['Churn'] == '1'][feature].dropna(), color= 'orange', label= 'Churn: Yes')
    
kdeplot('tenure')
kdeplot('MonthlyCharges')
kdeplot('TotalCharges')

#### 2.5 상관관계 확인

In [ ]:
relation=df.corr()
relation

In [ ]:
train=df.head(5001)
test=df.tail(2032)

# train.to_csv("train.csv",index=False)
# test.to_csv("test.csv",index=False)

test2=test[["tenure",'InternetService']]

### 3. 데이터 모델링  
  #### 3.1 의사결정나무 모델 적용  

In [ ]:
# predictors = ['SeniorCitizen', 'Partner', 'Dependents','tenure','InternetService','OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport','StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling','PaymentMethod']
predictors=["tenure",'InternetService']
 
x = train[predictors]
y = train["Churn"]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

decision = DecisionTreeClassifier().fit(x,y)
print(decision)

#### 3.2 랜덤포레스트 적용

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_jobs=-1).fit(x,y)
print(forest)

In [ ]:
from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold

decision_score = cross_val_score(decision, x, y, cv=5).mean()
forest_score = cross_val_score(forest, x, y, cv=5).mean()


print("DecisionTree = {0:.6f}".format(decision_score))
print("RandomForest = {0:.6f}".format(forest_score))

In [ ]:
prediction=decision.predict(test2)
submission = pd.DataFrame({
        "customerID": test["customerID"],
        "Churn_test": test["Churn"],
        "Churn_predict": prediction
    })
submission.loc[submission['Churn_predict']==submission['Churn_test'],'Predict']=1
submission.loc[submission['Churn_predict']!=submission['Churn_test'],'Predict']=0

# submission.to_csv('submission.csv', index=False)

print("predict percent is %.2f %%"%(sum(submission['Predict'])/len(submission)*100))

#### 3.3 로지스틱 회귀모델

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.cross_validation import train_test_split 

X=df[["MonthlyCharges","TotalCharges",'tenure']]
Y=df['Churn'] 

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

log_clf = LogisticRegression()
log_clf.fit(X_train,Y_train)
log_clf.score(X_test, Y_test)

In [ ]:
test3=test[["MonthlyCharges","TotalCharges","tenure"]]
# test3=test["MonthlyCharges"]

p=log_clf.predict(test3)
submission_log = pd.DataFrame({
        "customerID": test["customerID"],
        "Churn_test": test["Churn"],
        "Churn_predict": prediction
    })
submission_log.loc[submission['Churn_predict']==submission['Churn_test'],'Predict']=1
submission_log.loc[submission['Churn_predict']!=submission['Churn_test'],'Predict']=0

# submission_log.to_csv('submission_log.csv', index=False)

print("predict percent is %.2f %%"%(sum(submission_log['Predict'])/len(submission)*100))

### 4. 평가


의사결정나무 : 78.98%  
랜덤포레스트 : 78.42%  
로지스틱회귀 : 79.43%  

**-> 주어진 데이터 셋에서 로지스틱회귀 모델이 79.43%로 가장 높은 예측률을 보였으나 세 개의 모델 모두 78~80% 사이의 예측률을 보임 **